In [3]:
import cv2 as cv
import numpy as np

In [6]:
# SIFT 특징점 검출

img = cv.imread('./opencv_data/mot_color70.jpg') # 이미지 읽기

# gray 영상으로 변환
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# SIFT 적용
sift = cv.SIFT_create()
kp, des = sift.detectAndCompute(gray, None)

gray = cv.drawKeypoints(gray, kp, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv.imshow('shif 전 gray : ', gray)

cv.waitKey()
cv.destroyAllWindows()

In [1]:
%pip install opencv-python -U --force-reinstall

  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---------------------- ----------------- 7.1/12.6 MB 33.4 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 33.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.11.0.86
    Uninstalling opencv-python-4.11.0.86:
      Successfully uninstalled opencv-python-4.11.0.86
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.3 which is incompatible.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# SIFT 특징점 검출2

img = cv.imread('./opencv_data/mot_color70.jpg') # 이미지 읽기
img2 = cv.imread('./opencv_data/mot_color83.jpg') # 이미지 읽기

# gray 영상으로 변환
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
gray2 = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)

# SIFT 적용
sift = cv.SIFT_create()
kp, des = sift.detectAndCompute(gray, None)
kp2, des2 = sift.detectAndCompute(gray2, None)

gray = cv.drawKeypoints(gray, kp, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
gray2 = cv.drawKeypoints(gray2, kp2, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

cv.imshow('shit : ', gray)
cv.imshow('shit2 : ', gray2)

cv.waitKey()
cv.destroyAllWindows()

In [28]:
import cv2 as cv
import numpy as np
import time

# 이미지 로드
img1=cv.imread('./opencv_data/mot_color70.jpg')[190:350,440:560] # 버스를 크롭하여 모델 영상으로 사용
gray1=cv.cvtColor(img1,cv.COLOR_BGR2GRAY)
img2=cv.imread('./opencv_data/mot_color83.jpg')              # 장면 영상
gray2=cv.cvtColor(img2,cv.COLOR_BGR2GRAY)

sift=cv.SIFT_create()
kp1,des1=sift.detectAndCompute(gray1,None)
kp2,des2=sift.detectAndCompute(gray2,None)
print('특징점 개수:',len(kp1),len(kp2)) 

start=time.time()   # 시간 측정
# flann 매칭 알고리즘 사용
flann_matcher=cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
knn_match=flann_matcher.knnMatch(des1,des2,2)

# 임계값 지정
T=0.7
good_match=[]
for nearest1,nearest2 in knn_match:
    if (nearest1.distance/nearest2.distance)<T:
        good_match.append(nearest1)
print('매칭에 걸린 시간:',time.time()-start) 

img_match=np.empty((max(img1.shape[0],img2.shape[0]),img1.shape[1]+img2.shape[1],3),dtype=np.uint8)
cv.drawMatches(img1,kp1,img2,kp2,good_match,img_match,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv.imshow('Good Matches', img_match)

k=cv.waitKey()
cv.destroyAllWindows()

특징점 개수: 231 4096
매칭에 걸린 시간: 0.04218411445617676


In [49]:
import cv2 as cv
import numpy as np

img1=cv.imread('./opencv_data/mot_color70.jpg')[190:350,440:560] # 버스를 크롭하여 모델 영상으로 사용
gray1=cv.cvtColor(img1,cv.COLOR_BGR2GRAY)
img2=cv.imread('./opencv_data/mot_color83.jpg')			     # 장면 영상
gray2=cv.cvtColor(img2,cv.COLOR_BGR2GRAY)

# SIFT 작업 진행
sift=cv.SIFT_create()
kp1,des1=sift.detectAndCompute(gray1,None)
kp2,des2=sift.detectAndCompute(gray2,None)

# 매처를 이용하여 맵핑
flann_matcher=cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
# knnMatch() 함수는 각 특징점에 대해 가장 가까운 k개의 특징점을 찾아줌
knn_match=flann_matcher.knnMatch(des1,des2,2)	# 최근접 2개

# 임계값 설정
T=0.7
good_match=[]
for nearest1,nearest2 in knn_match:
    if (nearest1.distance/nearest2.distance)<T:
        good_match.append(nearest1)

# 호모그래피 행렬을 위한 작업
points1=np.float32([kp1[gm.queryIdx].pt for gm in good_match])
points2=np.float32([kp2[gm.trainIdx].pt for gm in good_match])
# RANSAC 방법으로 호모그래피 행렬 계산
H,_=cv.findHomography(points1,points2,cv.RANSAC)

h1,w1=img1.shape[0],img1.shape[1] 		# 첫 번째 영상의 크기
h2,w2=img2.shape[0],img2.shape[1] 		# 두 번째 영상의 크기

# box1은 버스 이미지 크기의 박스
box1=np.float32([[0,0],[0,h1-1],[w1-1,h1-1],[w1-1,0]]).reshape(4,1,2)
box2=cv.perspectiveTransform(box1,H)

# 위에서 얻어진 값을 통해서 img2에 박스를 그림
img2=cv.polylines(img2,[np.int32(box2)],True,(0,255,0),6)

img_match=np.empty((max(h1,h2),w1+w2,3),dtype=np.uint8)
cv.drawMatches(img1,kp1,img2,kp2,good_match,img_match,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
   
cv.imshow('Matches and Homography',img_match)

k=cv.waitKey()
cv.destroyAllWindows()

In [1]:
%pip install pyQt5

   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 6.9/6.9 MB 42.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.1 MB ? eta -:--:--
   -------- ------------------------------- 10.5/50.1 MB 131.1 MB/s eta 0:00:01
   --------- ------------------------------ 11.5/50.1 MB 27.8 MB/s eta 0:00:02
   ----------------- ---------------------- 22.0/50.1 MB 34.8 MB/s eta 0:00:01
   --------------------- ------------------ 26.7/50.1 MB 32.0 MB/s eta 0:00:01
   ---------------------- ----------------- 28.3/50.1 MB 33.2 MB/s eta 0:00:01
   ---------------------------------- ----- 43.0/50.1 MB 35.1 MB/s eta 0:00:01
   ------------------------------------- -- 46.9/50.1 MB 32.8 MB/s eta 0:00:01
   ---------------------------------------- 50.1/50.1 MB 30.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
